In [ ]:
import math
import numpy
import ipywidgets
from matplotlib import pyplot
%matplotlib inline

In [ ]:
import sys
modules_dir = '../src/python'
if modules_dir not in sys.path:
    sys.path.insert(0, modules_dir)

from regularize import regularize

In [ ]:
# Set the type and size of the font to use in Matplotlib figures.
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

In [ ]:
def ellipse(a, b, center=(0.0, 0.0), num=50):
    """
    Returns the coordinates of an ellipse.
    
    Parameters
    ----------
    a: float
        The semi-major axis of the ellipse.
    b: float
        The semi-minor axis of the ellipse.
    center: 2-tuple of floats, optional
        The position of the center of the ellipse;
        default: (0.0, 0.0)
    num: integer, optional
        The number of points on the upper side of the ellipse.
        The number includes the leading and trailing edges.
        Thus, the total number of points will be 2 * (num - 1);
        default: 50.
    
    Returns
    -------
    x: numpy.ndarray of floats
        The x-coordinates of the ellipse.
    y: numpy.ndarray of floats
        The y-coordinates of the ellipse.
    """
    xc, yc = center
    x_upper = numpy.linspace(xc + a, xc - a, num=num)
    y_upper = b / a * numpy.sqrt(a**2 - x_upper**2)
    x_lower = numpy.linspace(xc - a, xc + a, num=num)[1:-1]
    y_lower = -b / a * numpy.sqrt(a**2 - x_lower**2)
    x = numpy.concatenate((x_upper, x_lower))
    y = numpy.concatenate((y_upper, y_lower))
    return x, y

In [ ]:
def rotate(x, y, center=(0.0, 0.0), angle=0.0):
    """
    Applies a rotation around a center to given coordinates.
    
    Parameters
    ----------
    x: numpy.ndarray of floats
        The x-coordinates to rotate.
    y: numpy.ndarray of floats
        The y-coordinates to rotate.
    center: 2-tuple of floats, optional
        The center of rotation;
        default: (0.0, 0.0).
    angle: float, optional
        The angle of rotation in radians;
        default: 0.0.
    
    Returns
    -------
    x_new: numpy.ndarray of floats
        The rotated x-coordinates.
    y_new: numpy.ndarray of floats
        The rotated y-coordinates.
    """
    xc, yc = center
    x_new = xc + (x - xc) * numpy.cos(angle) - (y - yc) * numpy.sin(angle)
    y_new = yc + (x - xc) * numpy.sin(angle) + (y - yc) * numpy.cos(angle)
    return x_new, y_new

In [ ]:
class Flapping(object):
    def __init__(self, c=1.0, f=0.25, A0=2.8,
                 beta=numpy.pi / 4, alpha0=numpy.pi / 2, phi=0):
        self.c = c
        self.f = f
        self.A0 = A0
        self.beta = beta
        self.alpha0 = alpha0
        self.phi = phi
        self.Umax = numpy.pi * f * A0
    
    def orientation_angle(self, t):
        alpha = (self.alpha0 +
                 self.beta * numpy.sin(2 * numpy.pi * self.f * t +
                                       self.phi))
        return alpha
    
    def displacement(self, t):
        xd = self.A0 / 2 * numpy.cos(2 * numpy.pi * self.f * t)
        yd = 0.0
        return xd, yd
    
    def translational_velocity(self, t):
        ux = (-numpy.pi * self.f * self.A0 *
              numpy.sin(2 * numpy.pi * self.f * t))
        uy = 0.0 if isinstance(t, float) else numpy.zeros_like(t)
        return ux, uy
    
    def angular_velocity(self, t):
        w = (2 * numpy.pi * self.f * self.beta *
             numpy.cos(2 * numpy.pi * self.f * t + self.phi))
        return w
    
    def quasi_steady_coefficients(self, t):
        alpha = self.orientation_angle(t)
        U0, _ = self.translational_velocity(t)
        if U0 <= 0:
            alpha = numpy.pi - alpha
        CD = 1.4 - numpy.cos(2 * alpha)
        CL = 1.2 * numpy.sin(2 * alpha)
        return CD, CL
    
    def velocity(self, t, x, y, xc, yc):
        U0, V0 = self.translational_velocity(t)
        W0 = self.angular_velocity(t)
        ux, uy = U0 - W0 * (y - yc), V0 + W0 * (x - xc)
        return ux, uy
    
    def quasi_steady_forces(self, t, x, y, xc, yc, rho=1.0):
        CD, CL = self.quasi_steady_coefficients(t)
        xd, yd = self.displacement(t)
        ux, uy = self.velocity(t, x, y, xc, yc)
        u = numpy.sqrt(ux**2 + uy**2)
        D, L = 0.5 * rho * u**2 * CD, 0.5 * rho * u**2 * CL
        return D, L

In [ ]:
def kinematics(x0, y0, center, t, flapping):
    alpha = flapping.orientation_angle(t)
    x, y = rotate(x0, y0, center=center, angle=alpha)
    xd, yd = flapping.displacement(t)
    x += xd
    y += yd
    return x, y

In [ ]:
# Create an ellipse.
c = 1.0  # chord of the ellipse (major axis)
r = 0.1  # ratio between minor and major axis
a, b = c / 2.0, r * c / 2.0
x0, y0 = ellipse(a, b, center=(0.0, 0.0), num=100)
ds = 0.025  # target distance between two consecutive points.
x0, y0 = regularize(x0, y0, ds=ds)

# Plot the ellipse.
fig, ax = pyplot.subplots(figsize=(6.0, 6.0))
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.grid()
ax.plot(x0, y0, color='C0', linestyle='-', linewidth=2)
ax.axis('scaled', adjustable='box')
ax.set_xlim(-0.75, 0.75)
ax.set_ylim(-0.25, 0.25);

In [ ]:
def plot_position(t):
    flapping = Flapping()
    t /= flapping.f
    x, y, = kinematics(x0, y0, (0.0, 0.0), t, flapping)
    fig, (ax1, ax2) = pyplot.subplots(nrows=2, figsize=(8.0, 8.0))
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.grid()
    ax1.plot(x, y, color='C0', linestyle='-', linewidth=2)
    ax1.axis('scaled', adjustable='box')
    ax1.set_xlim(-2.0, 2.0)
    ax1.set_ylim(-1.0, 1.0)
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    xc, yc = flapping.displacement(t)
    x, y = x - xc, y - yc
    alpha = flapping.orientation_angle(t)
    D, L = flapping.quasi_steady_forces(t, x, y, 0.0, 0.0)
    U0, _ = flapping.translational_velocity(t + 1e-6)
    sign = 1.0 if U0 <= 0 else -1.0
    s = 4
    ax2.plot(x, y, color='C0')
    ax2.quiver(x[::s], y[::s], sign * D[::s], L[::s])
    ax2.axis('scaled', adjustable='box')
    ax2.set_xlim(-0.5, 0.5)
    ax2.set_ylim(-0.75, 0.75)
    fig.tight_layout()

In [ ]:
# Create and display an interative plot of the kinematics.
time_slider = ipywidgets.FloatSlider(value=0.0, min=0.0, max=4.0,
                                     step=0.05, description='t / T')
w = ipywidgets.interactive(plot_position, t=time_slider)
display(w)